# HuggingFace LLM inference experiments

- Compare inference options for quantisation and flash-atten
- As of 01/2024 torch's built in sdpa attention impmenentation has flash-attention without the need for the external lib from Tri Dao (which can be a pain to install). But it's not currently implemented within HF for all models -- particularly there's no support for Mistral
- So still need to use flash-atten lib for now to get flash-attention v2

In [1]:
import json
import os
import sys
from typing import List, Dict, Any
import requests

In [2]:
sys.path.append(os.getcwd())

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

First just load Mistral model with no additional options -- would be just as easy to use their reference implementation but HF gives all the other options for quant etc so go with this

In [4]:
del model

NameError: name 'model' is not defined

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.float16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████| 3/3 [00:06<00:00,  2.31s/it]


In [19]:
prompt = "[INST] What is your favourite condiment? [/INST] "
inputs = tokenizer(prompt, return_tensors="pt").to(device)
print(inputs)
generated_ids = model.generate(**inputs, max_new_tokens=256, do_sample=True)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'input_ids': tensor([[    1,   733, 16289, 28793,  1824,   349,   574, 16020,  2076,  2487,
         28804,   733, 28748, 16289, 28793, 28705]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
["[INST] What is your favourite condiment? [/INST] I don't have a favourite condiment as I don't consume food or condiments. However, some common favourites among people include ketchup, mayonnaise, hot sauce, soy sauce, and mustard."]


In [30]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

# encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(device)
print(encodeds)
generated_ids = model.generate(encodeds, max_new_tokens=256, do_sample=True)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1,   733, 16289, 28793,  1824,   349,   574, 16020,  2076,  2487,
         28804,   733, 28748, 16289, 28793,  6824, 28725,   315, 28742, 28719,
          3448, 10473,   298,   264,  1179, 11322, 19961,   302,  6138, 23598,
         18342, 28723,   661, 13633,   776,   272,  1103,  3558,   302,   686,
         16944, 15637,   423,   298,  5681,   315, 28742, 28719, 13198,   582,
           297,   272,  6132, 28808,     2,   733, 16289, 28793,  2378,   368,
           506,   993,  7136,   864, 21116, 28804,   733, 28748, 16289, 28793]],
       device='cuda:0')
["[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! [INST] Do you have mayonnaise recipes? [/INST] Yes, I have several mayonnaise recipes I'd be happy to share with you! Here's a classic one:\n\n**Classic Mayonnaise**\n\n* 1 egg yolk\n* 1 tbsp Dijon mustard\n* 1 cup ve

In [31]:
model.hf_device_map

{'': device(type='cuda')}